In [ ]:
#default_exp rescore.ML_score

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from numba import njit
import numpy as np
import pandas as pd
from tqdm import tqdm
        


        

In [ ]:
@njit
def fdr_to_q_values(fdr_values):
    q_values = np.zeros_like(fdr_values)
    min_q_value = np.max(fdr_values)
    for i in range(len(fdr_values) - 1, -1, -1):
        fdr = fdr_values[i]
        if fdr < min_q_value:
            min_q_value = fdr
        q_values[i] = min_q_value
    return q_values

def get_q_values(_df, score_column, decoy_column='decoy'):
    _df = _df.reset_index()
    _df = _df.sort_values([score_column,score_column], ascending=False)
    target_values = 1-_df[decoy_column].values
    decoy_cumsum = np.cumsum(_df[decoy_column].values)
    target_cumsum = np.cumsum(target_values)
    fdr_values = decoy_cumsum/target_cumsum
    _df['q_value'] = fdr_to_q_values(fdr_values)
    return _df

@njit
def score_to_q_value(scores, base_scores, base_q_values):
    q_values = np.zeros_like(scores)
    i,j = 0,0
    while i < len(scores) and j < len(base_scores):
        if scores[i] >= base_scores[j]:
            q_values[i] = base_q_values[j]
            i += 1
        else:
            j += 1
    while i < len(scores):
        q_values[i] = base_q_values[-1]
        i += 1
    return q_values

class Percolator:
    def __init__(self,
        cv_fold = 1,
        n_iter = 5,
        score_model=LogisticRegression,
        **sklearn_kwargs
    ):
        if score_model == LogisticRegression:
            self.model = LogisticRegression(solver='liblinear', **sklearn_kwargs)
        else:
            self.model = score_model(**sklearn_kwargs)

    def _estimate_q_value(self, df, fdr_level='spectrum'):
        df = df.sort_values(['ML_score','decoy'], ascending=False)
        df = df.reset_index(drop=True)
        if fdr_level == 'spectrum':
            target_values = 1-df['decoy'].values
            decoy_cumsum = np.cumsum(df['decoy'].values)
            target_cumsum = np.cumsum(target_values)
            fdr_values = decoy_cumsum/target_cumsum
            df['q_value'] = fdr_to_q_values(fdr_values)
        else:
            if fdr_level == 'precursor':
                _df = df.groupby(['sequence','mods','mod_sites','charge','decoy'])['ML_score'].max()
            elif fdr_level == 'peptide':
                _df = df.groupby(['sequence','mods','mod_sites','decoy'])['ML_score'].max()
            else:
                _df = df.groupby(['sequence','decoy'])['ML_score'].max()
            _df = _df.reset_index()
            _df = _df.sort_values(['ML_score','decoy'], ascending=False)
            target_values = 1-_df['decoy'].values
            decoy_cumsum = np.cumsum(_df['decoy'].values)
            target_cumsum = np.cumsum(target_values)
            fdr_values = decoy_cumsum/target_cumsum
            _df['q_value'] = fdr_to_q_values(fdr_values)
            df['q_value'] = score_to_q_value(df['ML_score'].values, _df['ML_score'].values, _df['q_value'].values)
        return df

    def _extract_features(self, df, nce, instrument):
        logging.info('[AlphaDeep] Extracting percolator features ...')

        predictions = self._predict(df, nce, instrument)
        msms_idx = predict_types.index('msms')
        rt_idx = predict_types.index('rt')
        mob_idx = predict_types.index('mobility')

        real_nterm_idx = []
        real_cterm_idx = []
        for i,iontype in enumerate(self.ion_types):
            if iontype in 'abc':
                real_nterm_idx.extend([i*self.max_ion_charge+ch for ch in range(self.max_ion_charge)])
            elif iontype in 'xyz':
                real_cterm_idx.extend([i*self.max_ion_charge+ch for ch in range(self.max_ion_charge)])

        pred_mapped_idx = []
        real_mapped_idx = []
        pred_mapped_nterm_idx = []
        pred_mapped_cterm_idx = []
        real_mapped_nterm_idx = []
        real_mapped_cterm_idx = []
        model_max_charge = self.msms_model.model_settings['model']['max_ion_charge']
        min_max_charge = min(model_max_charge, self.max_ion_charge)
        for i,iontype in enumerate(self.msms_model.model_settings['model']['ion_types']):
            if iontype in self.ion_types:
                pred_idx = [i*model_max_charge+ch for ch in range(min_max_charge)]
                pred_mapped_idx.extend(pred_idx)
                ion_idx = self.ion_types.index(iontype)
                real_idx = [ion_idx*self.max_ion_charge+ch for ch in range(min_max_charge)]
                real_mapped_idx.extend(real_idx)
                if iontype in 'abc':
                    real_mapped_nterm_idx.extend(real_idx)
                    pred_mapped_nterm_idx.extend(pred_idx)
                elif iontype in 'xyz':
                    real_mapped_cterm_idx.extend(real_idx)
                    pred_mapped_cterm_idx.extend(pred_idx)

        nterm_ion_ratio_list = []
        cterm_ion_ratio_list = []
        median_mass_err_list = []
        quantile75_mass_err_list = []
        msms_sa_list = []
        nterm_sa_list = []
        cterm_sa_list = []
        delta_rt_list = []

        for raw_name, scan, peptide, modinfo, charge in tqdm(df['raw_name,scan,peptide,modinfo,charge'.split(',')].values):
            spec_masses, spec_intens, rt, mob = self.ms_readers[raw_name][0].ReadMSInfoByScan(scan)
            intens, mass_errs = extract_psm_intens_and_masserrs(spec_masses, spec_intens, peptide, modinfo, self.ion_types, self.max_ion_charge, self.tol, self.ppm)
            mass_errs = mass_errs.reshape(-1)
            mass_errs = mass_errs[mass_errs<=self.tol]/self.tol
            if len(mass_errs) > 0:
                median_err = np.median(mass_errs)
                quantile75_err = np.quantile(mass_errs, 0.75)
            else:
                median_err = 1
                quantile75_err = 1
            preds = predictions[f'{peptide}|{modinfo}|{charge}']
            pred_intens = preds[msms_idx]
            pred_rt = preds[rt_idx]
            sa = get_sa(intens[:,real_mapped_idx], pred_intens[:, pred_mapped_idx])
            nterm_sa = get_sa(intens[:,real_mapped_nterm_idx], pred_intens[:, pred_mapped_nterm_idx])
            cterm_sa = get_sa(intens[:,real_mapped_cterm_idx], pred_intens[:, pred_mapped_cterm_idx])
            nterm_ion_ratio = np.count_nonzero(np.sum(intens[:,real_nterm_idx],1))/intens.shape[0]
            cterm_ion_ratio = np.count_nonzero(np.sum(intens[:,real_cterm_idx],1))/intens.shape[0]
            nterm_ion_ratio_list.append(nterm_ion_ratio)
            cterm_ion_ratio_list.append(cterm_ion_ratio)
            median_mass_err_list.append(median_err)
            quantile75_mass_err_list.append(quantile75_err)
            msms_sa_list.append(sa)
            nterm_sa_list.append(nterm_sa)
            cterm_sa_list.append(cterm_sa)
            delta_rt_list.append(abs(rt - pred_rt)/60)
        df['n_AA'] = df['peptide'].str.len()/30.0
        df['n_mod'] = df['modinfo'].str.split(';').str.len()/3.0
        df.loc[df['modinfo'].str.len()==0,'n_mod'] = 0
        df['cterm_ion_ratio'] = cterm_ion_ratio_list
        df['nterm_ion_ratio'] = nterm_ion_ratio_list
        df['median_ms2_mass_err'] = median_mass_err_list
        df['quantile75_ms2_mass_err'] = quantile75_mass_err_list
        df['msms_sa'] = msms_sa_list
        df['cterm_ion_sa'] = cterm_sa_list
        df['nterm_ion_sa'] = nterm_sa_list
        df['delta_rt'] = delta_rt_list
        return df

    def _cv_score(self, df):
        df = df.sample(frac=1).reset_index(drop=True)
        df_target = df[df['decoy'] == 0]
        df_decoy = df[df['decoy'] == 1]
        
        if self.cv_fold > 1:
            test_df_list = []
            for i in range(self.cv_fold):
                t_mask = np.ones(len(df_target), dtype=bool)
                _slice = slice(i, len(df_target), self.cv_fold)
                t_mask[_slice] = False
                cv_df_target = df_target[t_mask]
                train_t_df = cv_df_target[cv_df_target['q_value'] <= self.fdr]
                test_t_df = df_target[_slice]
                
                d_mask = np.ones(len(df_decoy), dtype=bool)
                _slice = slice(i, len(df_decoy), self.cv_fold)
                d_mask[_slice] = False
                train_d_df = df_decoy[d_mask]
                test_d_df = df_decoy[_slice]

                train_df = pd.concat((train_t_df, train_d_df))
                train_label = np.ones(len(train_df))
                train_label[len(train_t_df):] = 0
                test_df = pd.concat((test_t_df, test_d_df))

                if self.model == LogisticRegression:
                    _model = self.model(solver='liblinear')
                else:
                    _model = self.model()
                _model.fit(train_df[self.feature_list].values, train_label)
                if settings['percolator']['model'] in ['svm','lr']:
                    test_df['ML_score'] = _model.decision_function(test_df[self.feature_list].values)
                else:
                    test_df['ML_score'] = _model.predict_proba(test_df[self.feature_list].values)[:,1]
                test_df_list.append(test_df)
        
            return pd.concat(test_df_list)
        else:
            train_t_df = df_target[df_target['q_value'] <= self.fdr]

            train_df = pd.concat((train_t_df, df_decoy))
            train_label = np.ones(len(train_df))
            train_label[len(train_t_df):] = 0
            test_df = pd.concat((df_target, df_decoy))

            if self.model == LogisticRegression:
                _model = self.model(solver='liblinear')
            else:
                _model = self.model()
            _model.fit(train_df[self.feature_list].values, train_label)
            if settings['percolator']['model'] in ['svm','lr']:
                test_df['ML_score'] = _model.decision_function(test_df[self.feature_list].values)
            else:
                test_df['ML_score'] = _model.predict_proba(test_df[self.feature_list].values)[:,1]
        
            return test_df

    def alphapept_score(self, df, nce, instrument):
        from alphapept.score import filter_with_ML, train_RF
        exclude_features = ['sequence', 'precursor', 'raw_name', 'rt', 'scan', 'mobility',
                'peptide', 'modinfo', 'decoy','query_idx', 'db_idx','naked_sequence']
        df = self._extract_features(df, nce, instrument)
        cv, features = train_RF(df, exclude_features)
        df = filter_with_ML(df, cv, features = features)
        df['ML_score'] = df['score']
        return df

    def re_score(self, df, nce, instrument):
        df = self._extract_features(df, nce, instrument)
        df['ML_score'] = df['score']
        df = self._estimate_q_value(df, self.fdr_level)
        logging.info(f'{len(df[(df["q_value"]<=self.fdr) & (df["decoy"]==0)])} target PSMs at {self.fdr} spectrum-level FDR')
        for i in range(self.n_iter):
            logging.info(f'Iteration {i+1} of Percolator ...')
            df = self._cv_score(df)
            df = self._estimate_q_value(df)
            logging.info(f'{len(df[(df["q_value"]<=self.fdr) & (df["decoy"]==0)])} target PSMs at {self.fdr} spectrum-level FDR')
        df = self._estimate_q_value(df, self.fdr_level)
        logging.info(f'{len(df[(df["q_value"]<=self.fdr) & (df["decoy"]==0)])} target PSMs at {self.fdr} {self.fdr_level}-level FDR')
        return df